In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
import sklearn

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
df_anime = pd.read_csv('/Users/adithyaabhishek/Desktop/USML/project/report/data/anime.csv')
df_rating = pd.read_csv('/Users/adithyaabhishek/Desktop/USML/project/report/data/rating_complete.csv')

In [4]:
unknown_index = df_anime[df_anime['Episodes']=='Unknown'].index.to_list()

In [5]:
df_anime.loc[unknown_index,'Episodes'] = 0
df_anime[df_anime['Episodes']==0]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
11,21,One Piece,8.52,"Action, Adventure, Comedy, Super Power, Drama,...",One Piece,ONE PIECE,TV,0,"Oct 20, 1999 to ?",Fall 1999,...,292445.0,166186.0,141755.0,85424.0,35342.0,19019.0,8201.0,3675.0,2616.0,6340.0
211,235,Detective Conan,8.16,"Adventure, Mystery, Comedy, Police, Shounen",Case Closed,名探偵コナン,TV,0,"Jan 8, 1996 to ?",Winter 1996,...,27670.0,25632.0,31481.0,21333.0,7803.0,3401.0,993.0,415.0,254.0,469.0
871,966,Crayon Shin-chan,7.69,"Slice of Life, Comedy, Ecchi, School, Seinen",Shin Chan,クレヨンしんちゃん,TV,0,"Apr 13, 1992 to ?",Spring 1992,...,5422.0,4655.0,6762.0,6457.0,2788.0,1314.0,501.0,256.0,216.0,325.0
1093,1199,Nintama Rantarou,7.03,"Comedy, Kids",Unknown,忍たま乱太郎,TV,0,"Apr 10, 1993 to ?",Spring 1993,...,209.0,202.0,311.0,515.0,315.0,129.0,41.0,21.0,9.0,34.0
1787,1960,Sore Ike! Anpanman,6.38,"Comedy, Kids, Fantasy",Unknown,それいけ！アンパンマン,TV,0,"Oct 3, 1988 to ?",Fall 1988,...,82.0,57.0,80.0,163.0,163.0,118.0,29.0,19.0,16.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",Unknown,盗墓笔记之秦岭神树,ONA,0,"Apr 4, 2021 to ?",Unknown,...,Unknown,Unknown,Unknown,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17558,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",Unknown,見える子ちゃん,TV,0,2021 to ?,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17559,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,ひぐらしのなく頃に卒,TV,0,"Jul, 2021 to ?",Summer 2021,...,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
17560,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",Unknown,ヤマノススメ Next Summit,TV,0,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [6]:
df_anime['Episodes'] = df_anime['Episodes'].astype('int')

In [7]:
df_anime.loc[unknown_index,'Episodes'] = df_anime['Episodes'].median()

In [8]:
anime_index = {df_anime.loc[idx,'MAL_ID']:idx for idx in df_anime.index}

In [9]:
df_anime.head(5)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [10]:
df_anime.drop(df_anime[df_anime['Score'] == 'Unknown'].index, inplace = True)
df_anime['Score'] = df_anime['Score'].astype(float)
df_anime['Score'].dtypes

dtype('float64')

In [11]:
df_anime_clusterize = df_anime.drop(['MAL_ID'],axis=1)

In [12]:
df_anime_clusterize = df_anime.drop(['Name'],axis=1)

In [13]:
df_anime_clusterize.head(5)

,MAL_ID,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,Victor Entertainment,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...",...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,"TV Tokyo, Dentsu",...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [14]:
df_anime_clusterize = pd.get_dummies(df_anime_clusterize)

In [15]:
df_anime_clusterize.head(5)

,MAL_ID,Score,Episodes,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,...,Score-1_971.0,Score-1_974.0,Score-1_975.0,Score-1_976.0,Score-1_98.0,Score-1_982.0,Score-1_99.0,Score-1_996.0,Score-1_998.0,Score-1_Unknown
0,1,8.78,26,39,1251960,61971,105808,718161,71513,26678,...,0,0,0,0,0,0,0,0,0,0
1,5,8.39,1,518,273145,1174,4143,208333,1935,770,...,0,0,0,0,0,0,0,0,0,0
2,6,8.24,26,201,558913,12944,29113,343492,25465,13925,...,0,0,0,0,0,0,0,0,0,0
3,7,7.27,26,1467,94683,587,4300,46165,5121,5378,...,0,0,0,0,0,0,0,0,0,0
4,8,6.98,52,4369,13224,18,642,7314,766,1108,...,0,0,0,0,0,0,0,0,0,0


In [16]:
num_cols= df_anime_clusterize[['Episodes','Score','Members']]
cat_cols = df_anime_clusterize.drop(['Episodes','Score','Members'], axis=1)

In [17]:
scaler = StandardScaler()

In [18]:
num_cols = pd.DataFrame(scaler.fit_transform(num_cols))

In [19]:
num_cols.columns = ['episodes_scale','rating_scale','members_scale']

In [20]:
df_anime_clusterize = pd.concat([num_cols, cat_cols], axis=1, join='inner')

In [21]:
df_anime_clusterize_kmeans = df_anime_clusterize
df_anime_clusterize_mini_batch = df_anime_clusterize
df_anime_clusterize_agglo = df_anime_clusterize

## Kmeans

In [ ]:
from sklearn.cluster import KMeans
Kmeans = KMeans(n_clusters=9)
Kmeans.fit(df_anime_clusterize_kmeans)
scores_silhoutte_kmeans = sklearn.metrics.silhouette_score(df_anime_clusterize_kmeans, Kmeans.labels_)
scores_calinski_kmeans = sklearn.metrics.calinski_harabasz_score(df_anime_clusterize_kmeans, Kmeans.labels_)
scores_david_kmeans = sklearn.metrics.davies_bouldin_score(df_anime_clusterize_kmeans, Kmeans.labels_)


In [ ]:
clusters = Kmeans.predict(df_anime_clusterize_kmeans)
df_anime_clusterize_kmeans['cluster'] = clusters
df_anime_clusterize_kmeans['cluster'].value_counts()

rand_index_kmeans = sklearn.metrics.rand_score(Kmeans.labels_, clusters)
adjusted_rand1_kmeans = sklearn.metrics.adjusted_mutual_info_score(Kmeans.labels_, clusters)
print("Silhoutte Score :", scores_silhoutte_kmeans)
print("Calinski-Harabasz Index :",scores_calinski_kmeans)
print("Davies-Bouldin Index :",scores_david_kmeans)
print("Rand Index :",rand_index_kmeans)
print("Adjusted Rand Index :", adjusted_rand1_kmeans)

In [ ]:
plot_df = pd.DataFrame(np.array(df_anime_clusterize_kmeans.sample(4000)))
plot_df.columns = df_anime_clusterize_kmeans.columns
perplexity = 30
tsne_2d = TSNE(n_components=2, perplexity=perplexity)
TCs_2d = pd.DataFrame(tsne_2d.fit_transform(plot_df.drop(["cluster"], axis=1)))
TCs_2d.columns = ["TC1_2d","TC2_2d"]
plot_df = pd.concat([plot_df,TCs_2d], axis=1, join='inner')
plot_df["1d_y"] = 0
clusters = {}
for cluster_label in plot_df['cluster'].unique():
    clusters[cluster_label] = plot_df[plot_df["cluster"] == cluster_label]
data = []
for key in clusters.keys():
    data.append(go.Scatter(
                    x = clusters[key]["TC1_2d"],
                    y = clusters[key]["TC2_2d"],
                    mode = "markers",
                    name = "Cluster %s"%key,
                    text = None))

title = "Visualizing Clusters in Two Dimensions Using T-SNE (perplexity=" + str(perplexity) + ")"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)

iplot(fig)

## minibatch kmeans

In [24]:
from sklearn.cluster import MiniBatchKMeans
mini_batch =  MiniBatchKMeans(n_clusters=4,batch_size=40)
mini_batch = mini_batch.fit(df_anime_clusterize_mini_batch)
clusters = mini_batch.predict(df_anime_clusterize_mini_batch)
df_anime_clusterize_mini_batch['cluster'] = clusters
df_anime_clusterize_mini_batch['cluster'].value_counts()
scores_silhoutte_mini_batch = sklearn.metrics.silhouette_score(df_anime_clusterize_mini_batch, mini_batch.labels_)
scores_calinski_mini_batch = sklearn.metrics.calinski_harabasz_score(df_anime_clusterize_mini_batch, mini_batch.labels_)
scores_david_mini_batch = sklearn.metrics.davies_bouldin_score(df_anime_clusterize_mini_batch, mini_batch.labels_)
rand_index_mini_batch = sklearn.metrics.rand_score(mini_batch.labels_, clusters)
adjusted_rand_mini_batch = sklearn.metrics.adjusted_mutual_info_score(mini_batch.labels_, clusters)
print("Silhoutte Score :", scores_silhoutte_mini_batch)
print("Calinski-Harabasz Index :",scores_calinski_mini_batch)
print("Davies-Bouldin Index :",scores_david_mini_batch)
print("Rand Index :",rand_index_mini_batch)
print("Adjusted Rand Index :", adjusted_rand_mini_batch)

Silhoutte Score : 0.4601321584675467
Calinski-Harabasz Index : 7177.077669084196
Davies-Bouldin Index : 0.7153425195237213
Rand Index : 1.0
Adjusted Rand Index : 1.0


In [25]:
plot_df = pd.DataFrame(np.array(df_anime_clusterize_mini_batch.sample(4000)))
plot_df.columns = df_anime_clusterize_mini_batch.columns
perplexity = 30
tsne_2d = TSNE(n_components=2, perplexity=perplexity)
TCs_2d = pd.DataFrame(tsne_2d.fit_transform(plot_df.drop(["cluster"], axis=1)))
TCs_2d.columns = ["TC1_2d","TC2_2d"]
plot_df = pd.concat([plot_df,TCs_2d], axis=1, join='inner')
plot_df["1d_y"] = 0
clusters = {}
for cluster_label in plot_df['cluster'].unique():
    clusters[cluster_label] = plot_df[plot_df["cluster"] == cluster_label]
data = []
for key in clusters.keys():
    data.append(go.Scatter(
                    x = clusters[key]["TC1_2d"],
                    y = clusters[key]["TC2_2d"],
                    mode = "markers",
                    name = "Cluster %s"%key,
                    text = None))

title = "Visualizing Clusters in Two Dimensions Using T-SNE (perplexity=" + str(perplexity) + ")"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

/Users/adithyaabhishek/opt/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/adithyaabhishek/opt/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



## AgglomerativeClustering

In [26]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

In [ ]:
agglo = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
agglo = agglo.fit(df_anime_clusterize_agglo)
clusters = agglo.fit_predict(df_anime_clusterize_agglo)
# clusters = agglo.predict(df_anime_clusterize_agglo)
df_anime_clusterize_agglo['cluster'] = clusters
df_anime_clusterize_agglo['cluster'].value_counts()
scores_silhoutte_agglo = sklearn.metrics.silhouette_score(df_anime_clusterize_agglo, agglo.labels_)
scores_calinski_agglo = sklearn.metrics.calinski_harabasz_score(df_anime_clusterize_agglo, agglo.labels_)
scores_david_agglo = sklearn.metrics.davies_bouldin_score(df_anime_clusterize_agglo, agglo.labels_)
rand_index_agglo = sklearn.metrics.rand_score(agglo.labels_, clusters)
adjusted_rand_agglo = sklearn.metrics.adjusted_mutual_info_score(agglo.labels_, clusters)
print("Silhoutte Score :", scores_silhoutte_agglo)
print("Calinski-Harabasz Index :",scores_calinski_agglo)
print("Davies-Bouldin Index :",scores_david_agglo)
print("Rand Index :",rand_index_agglo)
print("Adjusted Rand Index :", adjusted_rand_agglo)


In [ ]:
plot_df = pd.DataFrame(np.array(df_anime_clusterize_mini_batch.sample(4000)))
plot_df.columns = df_anime_clusterize_mini_batch.columns
perplexity = 30
tsne_2d = TSNE(n_components=2, perplexity=perplexity)
TCs_2d = pd.DataFrame(tsne_2d.fit_transform(plot_df.drop(["cluster"], axis=1)))
TCs_2d.columns = ["TC1_2d","TC2_2d"]
plot_df = pd.concat([plot_df,TCs_2d], axis=1, join='inner')
plot_df["1d_y"] = 0
clusters = {}
for cluster_label in plot_df['cluster'].unique():
    clusters[cluster_label] = plot_df[plot_df["cluster"] == cluster_label]
data = []
for key in clusters.keys():
    data.append(go.Scatter(
                    x = clusters[key]["TC1_2d"],
                    y = clusters[key]["TC2_2d"],
                    mode = "markers",
                    name = "Cluster %s"%key,
                    text = None))

title = "Visualizing Clusters in Two Dimensions Using T-SNE (perplexity=" + str(perplexity) + ")"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [ ]:
anime_clusters = {i: [] for i in range(4)}
for anime_id, c_pred in zip(df_anime['MAL_ID'], df_anime_clusterize_kmeans['cluster']):
    anime_clusters[c_pred] +=[anime_id]

In [ ]:
def find_user_centroid(data):
    data = data[data['cluster']==data['cluster'].mode()[0]]
    data = data.drop(['user_id','anime_id','cluster'], axis=1,errors='ignore')
    return pd.DataFrame(data.mean(axis=0)).T

In [ ]:
df_rating['mean_rating'] = df_rating.groupby('user_id')['rating'].transform('mean')
a = df_rating[df_rating['rating']>=df_rating['mean_rating']].apply(lambda x: 1,axis=1)
index_liked = a.index.to_list()

df_rating_liked = df_rating.iloc[index_liked,:]
df_rating_liked = df_rating_liked.drop(['rating','mean_rating'], axis=1)

data = df_rating_liked[:100000]
grouped = data.groupby('user_id')

In [ ]:
train_data = {'user_id': [],'anime_id': []}
test_data = {'user_id': [],'anime_id': []}
for name,group in grouped:
    if len(group)>1:
        
        train, test = train_test_split(group['anime_id'],test_size=0.2,random_state=42)

        train_data['user_id']+=[name for _ in range(len(train))]
        train_data['anime_id']+= list(train)

        test_data['user_id']+=[name for _ in range(len(test))]
        test_data['anime_id']+= list(test)
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)     
df_train = df_train.join(df_anime_clusterize_kmeans, how='inner') 
train_centroids = pd.DataFrame(columns = ['user_id']+list(df_anime_clusterize_kmeans.columns))

for name,group in df_train.groupby('user_id'):
    user_centroid = find_user_centroid(group)
    user_centroid['user_id'] = name
    user_centroid['cluster'] = group['cluster'].mode()[0]
    train_centroids = train_centroids.append(user_centroid,ignore_index=True)
result = {}
for user_id in train_centroids['user_id'][:10]:
    print('User id %s'%user_id)
    user = train_centroids[train_centroids['user_id']==user_id]
    result_dist = []
   
    for anime_id in anime_clusters[user['cluster'].iloc[0]]:
        #iterate by all points in cluster. find 10 closer points to user centroid
        
        anime_point = df_anime_clusterize_kmeans.loc[anime_index[anime_id],:].drop('cluster').to_numpy()
        
        result_dist.append((anime_id, np.linalg.norm(user.drop(['cluster','user_id'],axis=1)-anime_point)))
        
    
    result[user_id] = sorted(result_dist,key=lambda x: x[1])[:10]

In [ ]:
anime_clusters = {i: [] for i in range(4)}
for anime_id, c_pred in zip(df_anime['MAL_ID'], df_anime_clusterize_mini_batch['cluster']):
    anime_clusters[c_pred] +=[anime_id]


In [ ]:
train_data = {'user_id': [],'anime_id': []}
test_data = {'user_id': [],'anime_id': []}
for name,group in grouped:
    if len(group)>1:
        
        train, test = train_test_split(group['anime_id'],test_size=0.2,random_state=42)

        train_data['user_id']+=[name for _ in range(len(train))]
        train_data['anime_id']+= list(train)

        test_data['user_id']+=[name for _ in range(len(test))]
        test_data['anime_id']+= list(test)
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)     
df_train = df_train.join(df_anime_clusterize_mini_batch, how='inner') 
train_centroids = pd.DataFrame(columns = ['user_id']+list(df_anime_clusterize_mini_batch.columns))

for name,group in df_train.groupby('user_id'):
    user_centroid = find_user_centroid(group)
    user_centroid['user_id'] = name
    user_centroid['cluster'] = group['cluster'].mode()[0]
    train_centroids = train_centroids.append(user_centroid,ignore_index=True)
result = {}
for user_id in train_centroids['user_id'][:10]:
    print('User id %s'%user_id)
    user = train_centroids[train_centroids['user_id']==user_id]
    result_dist = []
   
    for anime_id in anime_clusters[user['cluster'].iloc[0]]:
        #iterate by all points in cluster. find 10 closer points to user centroid
        
        anime_point = df_anime_clusterize_mini_batch.loc[anime_index[anime_id],:].drop('cluster').to_numpy()
        
        result_dist.append((anime_id, np.linalg.norm(user.drop(['cluster','user_id'],axis=1)-anime_point)))
        
    
    result[user_id] = sorted(result_dist,key=lambda x: x[1])[:10]

In [ ]:
anime_clusters = {i: [] for i in range(4)}
for anime_id, c_pred in zip(df_anime['MAL_ID'], df_anime_clusterize_agglo['cluster']):
    anime_clusters[c_pred] +=[anime_id]

In [ ]:
train_data = {'user_id': [],'anime_id': []}
test_data = {'user_id': [],'anime_id': []}
for name,group in grouped:
    if len(group)>1:
        
        train, test = train_test_split(group['anime_id'],test_size=0.2,random_state=42)

        train_data['user_id']+=[name for _ in range(len(train))]
        train_data['anime_id']+= list(train)

        test_data['user_id']+=[name for _ in range(len(test))]
        test_data['anime_id']+= list(test)
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)     
df_train = df_train.join(df_anime_clusterize_agglo, how='inner') 
train_centroids = pd.DataFrame(columns = ['user_id']+list(df_anime_clusterize_agglo.columns))

for name,group in df_train.groupby('user_id'):
    user_centroid = find_user_centroid(group)
    user_centroid['user_id'] = name
    user_centroid['cluster'] = group['cluster'].mode()[0]
    train_centroids = train_centroids.append(user_centroid,ignore_index=True)
result = {}
for user_id in train_centroids['user_id'][:10]:
    print('User id %s'%user_id)
    user = train_centroids[train_centroids['user_id']==user_id]
    result_dist = []
   
    for anime_id in anime_clusters[user['cluster'].iloc[0]]:
        #iterate by all points in cluster. find 10 closer points to user centroid
        
        anime_point = df_anime_clusterize_agglo.loc[anime_index[anime_id],:].drop('cluster').to_numpy()
        
        result_dist.append((anime_id, np.linalg.norm(user.drop(['cluster','user_id'],axis=1)-anime_point)))
        
    
    result[user_id] = sorted(result_dist,key=lambda x: x[1])[:10]